In [1]:
import nltk
import string
from datasets import load_dataset
import re
from nltk.corpus import stopwords
from nltk import pos_tag
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from scipy.sparse import csr_matrix
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import mlflow
from sklearnex import patch_sklearn
from warnings import filterwarnings
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/uxlfoundation/scikit-learn-intelex)


In [2]:
filterwarnings("ignore")

In [3]:
ag_news_dataset = load_dataset("ag_news")
stop_words = set(stopwords.words("english"))

#### Нужно: (ничего, Стемминг, Лемматизация) * (binary, count, tf-idf)) -> F1_Macro

In [4]:
text_preprocess_types = [None, 'стемминг', 'лемматизация']
#words_classes = ['N', 'NJ', 'NJV', 'ALL']
words_classes = ['NJ', 'NJV', 'ALL']

#frequency_filtration_types = [None, 'low', 'high', 'both']
frequency_filtration_types = [None]

vector_representation_types = ['binary', 'count', 'tfidf']

In [5]:
iterations_num = len(text_preprocess_types) * len(words_classes) * len(frequency_filtration_types)
print(iterations_num)

24


In [6]:
def base_ag_news_preprocess(text):
    tokens = text.lower()

    # Удаление спец слов
    special_words = ['reuters', 'afp', 'ap', 'usatoday.com', 'forbes.com', 'target=/stocks/quickinfo/fullquote"' ]
    for word in special_words:
        tokens = tokens.replace(word, '')
    
    pattern = r'[&lt][^<>]*&gt'
    tokens = re.sub(pattern, '', tokens)
    
    # Удаление пунктуации и цифр
    #tokens = ''.join(i if i not in set(string.punctuation) - set('-') | set(string.digits) else ' ' for i in tokens)
    tokens = ''.join(i if i not in set(string.punctuation)  | set(string.digits) else ' ' for i in tokens)
    
    # Токенизация
    tokens = nltk.word_tokenize(tokens)
    
    # Удаление стоп слов
    #stop_wordsL = stop_words - {'no','not'}
    stop_wordsL = stop_words
    tokens = [word for word in tokens if (word not in stop_wordsL and word != '-')]
    return tokens

In [7]:
def different_ag_news_preprocess(tokens, preprocess_type, words_class):
    
    # Обработка слов
    if preprocess_type == 'лемматизация':
        lemmatizer = nltk.WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
    elif preprocess_type == 'стемминг':
        stemmer = nltk.PorterStemmer()
        tokens = [stemmer.stem(token) for token in tokens]
    
    # Обработка частей речи
    tokens = pos_tag(tokens)
    if words_class == 'N':
        tokens = [word for word, tag in tokens if tag.startswith('N')]
    elif words_class == 'NJ':
        tokens = [word for word, tag in tokens if tag.startswith('N') or tag.startswith('J')]
    elif words_class == 'NJV':
        tokens = [word for word, tag in tokens if tag.startswith('N') or tag.startswith('J') or tag.startswith('V')]
    elif words_class == 'ALL':
        tokens = [word for word, _ in tokens]
    
    return tokens  

In [8]:
def frequency_filtration(words_dictionary, frequency_filtration_type):
    if frequency_filtration_type == 'low':
        return dict([(key, value) for key, value in words_dictionary.items() if value >= 10 ])
    elif frequency_filtration_type == 'high':
        return dict([(key, value) for key, value in words_dictionary.items() if value <= 3000])
    elif frequency_filtration_type == 'both':
        return dict([(key, value) for key, value in words_dictionary.items() if 10 <= value <= 3000])
    else:
        return words_dictionary

In [9]:
def dummy(doc):
    return doc

In [178]:
def final_ag_news_preprocess(dataset, model_type):
    # Подготовка данных
    x_train = dataset['train']['text']
    y_train = dataset['train']['label']
    
    x_test = dataset['test']['text']
    y_test = dataset['test']['label']
    
    # Базовая обработка
    for i, text in enumerate(x_train):
        x_train[i] = base_ag_news_preprocess(text)
        
    for i, text in enumerate(x_test):
        x_test[i] = base_ag_news_preprocess(text)
    
    index = 2
    # Вариативная обработка
    for preprocess_type in text_preprocess_types: # 3 варианта
        for words_class in words_classes:         # 4 варианта
            words = {}
            xtr = x_train
            xte = x_test
            
            # Обработка текстов
            for i, tokens in enumerate(xtr):
                final_tokens = different_ag_news_preprocess(tokens, preprocess_type, words_class)
                xtr[i] = final_tokens
                
                # Заполнение словаря
                for token in final_tokens:
                    if token not in words:
                        words[token] = 1
                    else:
                        words[token] += 1
            
            xte = [different_ag_news_preprocess(tokens, preprocess_type, words_class) for tokens in xte]
                    
            # Фильтрация по частоте
            for frequency_filtration_type in frequency_filtration_types: # 4 варианта
                filtered_words = frequency_filtration(words, frequency_filtration_type)
                token_length = len(filtered_words)
                
                # Векторизация слов
                word_list = sorted(filtered_words.keys())
                # Присвоение словам индексов
                words_indexed = {}
                for idx, word in enumerate(word_list):
                    words_indexed[word] = idx
                
                # OHE
                vectorizer_OHE = CountVectorizer(vocabulary=words_indexed, tokenizer=dummy, preprocessor=dummy, dtype=np.int8, binary=True)
                x_train_OHE = vectorizer_OHE.fit_transform(xtr)
                x_test_OHE = vectorizer_OHE.transform(xte)
                
                # COUNT
                vectorizer_COUNT = CountVectorizer(vocabulary=words_indexed, tokenizer=dummy, preprocessor=dummy, dtype=np.int8)
                x_train_COUNT = vectorizer_COUNT.fit_transform(xtr)
                x_test_COUNT = vectorizer_COUNT.transform(xte)
                
                # TF-IDF
                vectorizer_TFIDF = TfidfVectorizer(vocabulary=words_indexed, preprocessor=dummy, tokenizer=dummy, dtype=np.float32)
                x_train_TFIDF = vectorizer_TFIDF.fit_transform(xtr)
                x_test_TFIDF = vectorizer_TFIDF.transform(xte)

                # Построение классификаторов
                if model_type == 'DT':
                    clf_OHE = DecisionTreeClassifier()
                    clf_COUNT = DecisionTreeClassifier()
                    clf_TFIDF = DecisionTreeClassifier()
                elif model_type == 'RF':
                    clf_OHE = RandomForestClassifier(n_estimators=50, n_jobs=-1)
                    clf_COUNT = RandomForestClassifier(n_estimators=50, n_jobs=-1)
                    clf_TFIDF = RandomForestClassifier(n_estimators=50, n_jobs=-1)
                elif model_type == 'ADA':
                    clf_OHE =  AdaBoostClassifier(estimator=RandomForestClassifier(n_estimators=12, max_depth=25, n_jobs=-1), n_estimators=50, learning_rate=0.3)
                    clf_COUNT =  AdaBoostClassifier(estimator=RandomForestClassifier(n_estimators=12, max_depth=25, n_jobs=-1), n_estimators=50, learning_rate=0.3)
                    clf_TFIDF =  AdaBoostClassifier(estimator=RandomForestClassifier(n_estimators=12, max_depth=25, n_jobs=-1), n_estimators=50, learning_rate=0.3)
                
                # Обучение классификаторов
                clf_OHE = clf_OHE.fit(x_train_OHE, y_train)
                clf_COUNT = clf_COUNT.fit(x_train_COUNT, y_train)
                clf_TFIDF = clf_TFIDF.fit(x_train_TFIDF, y_train)
                
                # Тестирование
                predictions_OHE = clf_OHE.predict(x_test_OHE)
                predictions_COUNT = clf_COUNT.predict(x_test_COUNT)
                predictions_TFIDF = clf_TFIDF.predict(x_test_TFIDF)
                
                # Метрики
                macro_score_OHE = f1_score(y_test, predictions_OHE, average='macro')
                macro_score_COUNT = f1_score(y_test, predictions_COUNT, average='macro')
                macro_score_TFIDF = f1_score(y_test, predictions_TFIDF, average='macro')
                
                mlflow.start_run(run_name=f'{model_type}_{preprocess_type}_{words_class}_{frequency_filtration_type}_OHE')
                mlflow.log_param('model', clf_OHE.__class__.__name__)
                mlflow.log_param('preprocess_type', preprocess_type)
                mlflow.log_param('words_class', words_class)
                mlflow.log_param('frequency_filtration_type', frequency_filtration_type)
                mlflow.log_param('token_length', token_length)
                mlflow.sklearn.log_model(clf_OHE, 'DecisionTreeClassifier')
                mlflow.log_metric('macro_score', macro_score_OHE)
                mlflow.end_run()
                
                mlflow.start_run(run_name=f'{model_type}_{preprocess_type}_{words_class}_{frequency_filtration_type}_COUNT')
                mlflow.log_param('model', clf_COUNT.__class__.__name__)
                mlflow.log_param('preprocess_type', preprocess_type)
                mlflow.log_param('words_class', words_class)
                mlflow.log_param('frequency_filtration_type', frequency_filtration_type)
                mlflow.log_param('token_length', token_length)
                mlflow.sklearn.log_model(clf_COUNT, 'DecisionTreeClassifier')
                mlflow.log_metric('macro_score', macro_score_COUNT)
                mlflow.end_run()
                
                mlflow.start_run(run_name=f'{model_type}_{preprocess_type}_{words_class}_{frequency_filtration_type}_TFIDF')
                mlflow.log_param('model', clf_TFIDF.__class__.__name__)
                mlflow.log_param('preprocess_type', preprocess_type)
                mlflow.log_param('words_class', words_class)
                mlflow.log_param('frequency_filtration_type', frequency_filtration_type)
                mlflow.log_param('token_length', token_length)
                mlflow.sklearn.log_model(clf_TFIDF, 'DecisionTreeClassifier')
                mlflow.log_metric('macro_score', macro_score_TFIDF)
                mlflow.end_run()
                
                index += 1
                print(f'Итерация {index} / {iterations_num}')

In [11]:
def one_run(dataset, preprocess_type, words_class, frequency_filtration_type, model_type):
    # Подготовка данных
    x_train = dataset['train']['text']
    y_train = dataset['train']['label']
    
    x_test = dataset['test']['text']
    y_test = dataset['test']['label']
    
    # Базовая обработка
    for i, text in enumerate(x_train):
        x_train[i] = base_ag_news_preprocess(text)
        
    for i, text in enumerate(x_test):
        x_test[i] = base_ag_news_preprocess(text)
    
    words = {}
    xtr = x_train
    xte = x_test
            
    # Обработка текстов
    for i, tokens in enumerate(xtr):
        final_tokens = different_ag_news_preprocess(tokens, preprocess_type, words_class)
        xtr[i] = final_tokens
                
        # Заполнение словаря
        for token in final_tokens:
            if token not in words:
                words[token] = 1
            else:
                words[token] += 1
            
    xte = [different_ag_news_preprocess(tokens, preprocess_type, words_class) for tokens in xte]
                    
    # Фильтрация по частоте
    filtered_words = frequency_filtration(words, frequency_filtration_type)
    token_length = len(filtered_words)
                
    # Векторизация слов
    word_list = sorted(filtered_words.keys())
    # Присвоение словам индексов
    words_indexed = {}
    for idx, word in enumerate(word_list):
        words_indexed[word] = idx
        
    # OHE
    vectorizer_OHE = CountVectorizer(vocabulary=words_indexed, tokenizer=dummy, preprocessor=dummy, dtype=np.int8, binary=True)
    x_train_OHE = vectorizer_OHE.fit_transform(xtr)
    x_test_OHE = vectorizer_OHE.transform(xte)
                
    # COUNT
    vectorizer_COUNT = CountVectorizer(vocabulary=words_indexed, tokenizer=dummy, preprocessor=dummy, dtype=np.int8)
    x_train_COUNT = vectorizer_COUNT.fit_transform(xtr)
    x_test_COUNT = vectorizer_COUNT.transform(xte)
                
    # TF-IDF
    vectorizer_TFIDF = TfidfVectorizer(vocabulary=words_indexed, preprocessor=dummy, tokenizer=dummy, dtype=np.float32)
    x_train_TFIDF = vectorizer_TFIDF.fit_transform(xtr)
    x_test_TFIDF = vectorizer_TFIDF.transform(xte)
    
    # Построение классификаторов
    if model_type == 'DT':
        clf_OHE = DecisionTreeClassifier()
        clf_COUNT = DecisionTreeClassifier()
        clf_TFIDF = DecisionTreeClassifier()
    elif model_type == 'RF':
        clf_OHE = RandomForestClassifier(n_estimators=50, n_jobs=-1)
        clf_COUNT = RandomForestClassifier(n_estimators=50, n_jobs=-1)
        clf_TFIDF = RandomForestClassifier(n_estimators=50, n_jobs=-1)
    
    # Обучение классификаторов
    clf_OHE = clf_OHE.fit(x_train_OHE, y_train)
    clf_COUNT = clf_COUNT.fit(x_train_COUNT, y_train)
    clf_TFIDF = clf_TFIDF.fit(x_train_TFIDF, y_train)
                
    # Тестирование
    predictions_OHE = clf_OHE.predict(x_test_OHE)
    predictions_COUNT = clf_COUNT.predict(x_test_COUNT)
    predictions_TFIDF = clf_TFIDF.predict(x_test_TFIDF)
                
    # Метрики
    macro_score_OHE = f1_score(y_test, predictions_OHE, average='macro')
    macro_score_COUNT = f1_score(y_test, predictions_COUNT, average='macro')
    macro_score_TFIDF = f1_score(y_test, predictions_TFIDF, average='macro')
    
    mlflow.start_run(run_name=f'test_{preprocess_type}_{words_class}_{frequency_filtration_type}_OHE')
    mlflow.log_param('model', clf_OHE.__class__.__name__)
    mlflow.log_param('tree_depth', clf_OHE.get_depth())
    mlflow.log_param('preprocess_type', preprocess_type)
    mlflow.log_param('words_class', words_class)
    mlflow.log_param('frequency_filtration_type', frequency_filtration_type)
    mlflow.log_param('token_length', token_length)
    mlflow.sklearn.log_model(clf_OHE, 'DecisionTreeClassifier')
    mlflow.log_metric('macro_score', macro_score_OHE)
    mlflow.end_run()
    
    mlflow.start_run(run_name=f'test_{preprocess_type}_{words_class}_{frequency_filtration_type}_COUNT')
    mlflow.log_param('model', clf_COUNT.__class__.__name__)
    mlflow.log_param('tree_depth', clf_TFIDF.get_depth())
    mlflow.log_param('preprocess_type', preprocess_type)
    mlflow.log_param('words_class', words_class)
    mlflow.log_param('frequency_filtration_type', frequency_filtration_type)
    mlflow.log_param('token_length', token_length)
    mlflow.sklearn.log_model(clf_COUNT, 'DecisionTreeClassifier')
    mlflow.log_metric('macro_score', macro_score_COUNT)
    mlflow.end_run()
    
    mlflow.start_run(run_name=f'test_{preprocess_type}_{words_class}_{frequency_filtration_type}_TFIDF')
    mlflow.log_param('model', clf_TFIDF.__class__.__name__)
    mlflow.log_param('tree_depth', clf_TFIDF.get_depth())
    mlflow.log_param('preprocess_type', preprocess_type)
    mlflow.log_param('words_class', words_class)
    mlflow.log_param('frequency_filtration_type', frequency_filtration_type)
    mlflow.log_param('token_length', token_length)
    mlflow.sklearn.log_model(clf_TFIDF, 'DecisionTreeClassifier')
    mlflow.log_metric('macro_score', macro_score_TFIDF)
    mlflow.end_run()
    

In [179]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment(experiment_name="agNewsADA")

2025/03/12 00:10:31 INFO mlflow.tracking.fluent: Experiment with name 'agNewsADA' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/793654491394688587', creation_time=1741727431630, experiment_id='793654491394688587', last_update_time=1741727431630, lifecycle_stage='active', name='agNewsADA', tags={}>

In [180]:
final_ag_news_preprocess(ag_news_dataset, model_type='ADA')

2025/03/12 00:14:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run ADA_None_N_None_OHE at: http://127.0.0.1:5000/#/experiments/793654491394688587/runs/2f9f6fa7205b4302bafd231eb383eb56
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/793654491394688587


2025/03/12 00:14:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run ADA_None_N_None_COUNT at: http://127.0.0.1:5000/#/experiments/793654491394688587/runs/ec6294b3c04c46d2ac594ce3b832cc5d
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/793654491394688587


2025/03/12 00:14:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

🏃 View run ADA_лемматизация_ALL_low_OHE at: http://127.0.0.1:5000/#/experiments/793654491394688587/runs/103cc6a184484693bbfbcd7aaf57a018
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/793654491394688587


2025/03/12 00:51:03 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run ADA_лемматизация_ALL_low_COUNT at: http://127.0.0.1:5000/#/experiments/793654491394688587/runs/1873838f0ce74569a3186c5ee45539e3
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/793654491394688587


2025/03/12 00:51:07 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run ADA_лемматизация_ALL_low_TFIDF at: http://127.0.0.1:5000/#/experiments/793654491394688587/runs/75580dcd2a114361921b0faaebbcd608
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/793654491394688587
Итерация 26 / 24


In [13]:
one_run(ag_news_dataset, 'лемматизация', 'NJ', 'low', 'RF')

2025/03/11 16:28:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run test_лемматизация_NJ_low_OHE at: http://127.0.0.1:5000/#/experiments/537408113752698407/runs/23ea94598c4143bfb298c45c0cf0974a
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/537408113752698407


2025/03/11 16:28:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run test_лемматизация_NJ_low_COUNT at: http://127.0.0.1:5000/#/experiments/537408113752698407/runs/9198223fdacc4638acea6f8344de3f75
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/537408113752698407


2025/03/11 16:29:02 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run test_лемматизация_NJ_low_TFIDF at: http://127.0.0.1:5000/#/experiments/537408113752698407/runs/4b6ae5d8cddc46358b79466b1a52e854
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/537408113752698407


In [16]:
cc = DecisionTreeClassifier()

In [21]:
dataset = ag_news_dataset
preprocess_type = 'лемматизация'
words_class = 'NJ'
frequency_filtration_type = 'low'

# Подготовка данных
x_train = dataset['train']['text']
y_train = dataset['train']['label']
    
x_test = dataset['test']['text']
y_test = dataset['test']['label']
    
# Базовая обработка
for i, text in enumerate(x_train):
    x_train[i] = base_ag_news_preprocess(text)
        
for i, text in enumerate(x_test):
    x_test[i] = base_ag_news_preprocess(text)
    
words = {}
xtr = x_train
xte = x_test
            
# Обработка текстов
for i, tokens in enumerate(xtr):
    final_tokens = different_ag_news_preprocess(tokens, preprocess_type, words_class)
    xtr[i] = final_tokens
                
    # Заполнение словаря
    for token in final_tokens:
        if token not in words:
            words[token] = 1
        else:
            words[token] += 1
            
xte = [different_ag_news_preprocess(tokens, preprocess_type, words_class) for tokens in xte]
                    
# Фильтрация по частоте
filtered_words = frequency_filtration(words, frequency_filtration_type)
token_length = len(filtered_words)
                
# Векторизация слов
word_list = sorted(filtered_words.keys())
# Присвоение словам индексов
words_indexed = {}
for idx, word in enumerate(word_list):
    words_indexed[word] = idx
    
# OHE
vectorizer_OHE = CountVectorizer(vocabulary=words_indexed, tokenizer=dummy, preprocessor=dummy, dtype=np.int8, binary=True)
x_train_OHE = vectorizer_OHE.fit_transform(xtr)
x_test_OHE = vectorizer_OHE.transform(xte)
            
# COUNT
vectorizer_COUNT = CountVectorizer(vocabulary=words_indexed, tokenizer=dummy, preprocessor=dummy, dtype=np.int8)
x_train_COUNT = vectorizer_COUNT.fit_transform(xtr)
x_test_COUNT = vectorizer_COUNT.transform(xte)
            
# TF-IDF
vectorizer_TFIDF = TfidfVectorizer(vocabulary=words_indexed, preprocessor=dummy, tokenizer=dummy, dtype=np.float32)
x_train_TFIDF = vectorizer_TFIDF.fit_transform(xtr)
x_test_TFIDF = vectorizer_TFIDF.transform(xte)

In [73]:
clf_OHE = RandomForestClassifier(n_estimators=50, n_jobs=-1)
clf_OHE = clf_OHE.fit(x_train_OHE, y_train)

In [74]:
predictions_OHE = clf_OHE.predict(x_test_OHE)
macro_score_OHE = f1_score(y_test, predictions_OHE, average='macro')

In [75]:
print(macro_score_OHE)

0.8811401647147361


In [12]:
clf_COUNT = RandomForestClassifier(n_estimators=50, n_jobs=-1)
#clf_COUNT = clf_COUNT.fit(x_train_COUNT, y_train)

In [79]:
predictions_COUNT = clf_COUNT.predict(x_test_OHE)
macro_score_COUNT = f1_score(y_test, predictions_COUNT, average='macro')

In [80]:
print(macro_score_COUNT)

0.8793384460507578


In [81]:
clf_TFIDF = RandomForestClassifier(n_estimators=50, n_jobs=-1)
clf_TFIDF = clf_TFIDF.fit(x_train_TFIDF, y_train)

In [82]:
predictions_TFIDF = clf_TFIDF.predict(x_test_TFIDF)
macro_score_TFIDF = f1_score(y_test, predictions_TFIDF, average='macro')

In [84]:
print(macro_score_TFIDF)

0.8800442265813493


In [72]:
clf_OHE_GB = GradientBoostingClassifier()
clf_OHE_GB = clf_OHE_GB.fit(x_train_OHE, y_train)

In [75]:
predictions_GB_OHE = clf_OHE_GB.predict(x_test_OHE)
macro_score_OHE_GB = f1_score(y_test, predictions_GB_OHE, average='macro')

In [76]:
print(macro_score_OHE_GB)

0.8202423408128906


In [174]:
clf_OHE_ADA = AdaBoostClassifier(estimator=RandomForestClassifier(n_estimators=12, max_depth=25, n_jobs=-1), n_estimators=50, learning_rate=0.3)
clf_OHE_ADA = clf_OHE_ADA.fit(x_train_OHE, y_train)

In [175]:
predictions_ADA_OHE = clf_OHE_ADA.predict(x_test_OHE)
macro_score_ADA_OHE = f1_score(y_test, predictions_ADA_OHE, average='macro')

In [176]:
print(macro_score_ADA_OHE)

0.8810723137667106


In [171]:
clf_COUNT_ADA = AdaBoostClassifier(estimator=RandomForestClassifier(n_estimators=12, max_depth=25, n_jobs=-1), n_estimators=50, learning_rate=0.3)
clf_COUNT_ADA = clf_OHE_ADA.fit(x_train_COUNT, y_train)

In [172]:
predictions_ADA_COUNT = clf_COUNT_ADA.predict(x_test_COUNT)
macro_score_ADA_COUNT = f1_score(y_test, predictions_ADA_COUNT, average='macro')

In [173]:
print(macro_score_ADA_COUNT)

0.8812967726235373


In [168]:
clf_TFIDF_ADA = AdaBoostClassifier(estimator=RandomForestClassifier(n_estimators=12, max_depth=25, n_jobs=-1), n_estimators=50, learning_rate=0.3)
clf_TFIDF_ADA = clf_OHE_ADA.fit(x_train_TFIDF, y_train)

In [169]:
predictions_ADA_TFIDF = clf_TFIDF_ADA.predict(x_test_TFIDF)
macro_score_ADA_TFIDF = f1_score(y_test, predictions_ADA_TFIDF, average='macro')

In [170]:
print(macro_score_ADA_TFIDF)

0.8857382437564187


In [177]:
clf_OHE_ADA.__class__.__name__

'AdaBoostClassifier'